## Audit Configuration

In [0]:
import dlt
import json
from pyspark.sql.functions import *
from pyspark.sql.types import *
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from pyspark.sql.window import Window
import uuid
from delta.tables import DeltaTable

In [0]:
# dbutils.fs.ls("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/JOH")

In [0]:
# audit_mnt = "/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/TD"
audit_delta_path = "/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/JOH/joh_cr_audit_table"

In [0]:


def datetime_uuid():
    dt_str = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, dt_str))

run_id_value = datetime_uuid()

audit_schema = StructType([
    StructField("Runid", StringType(), True),
    StructField("Unique_identifier_desc", StringType(), True),
    StructField("Unique_identifier", StringType(), True),
    StructField("Table_name", StringType(), True),
    StructField("Stage_name", StringType(), True),
    StructField("Record_count", IntegerType(), True),
    StructField("Run_dt", TimestampType(), True),
    StructField("Batch_id", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("File_name", StringType(), True),
    StructField("Status", StringType(), True)
])

def create_audit_df(df: DataFrame, unique_identifier_desc: str, table_name: str, stage_name: str, description: str, additional_columns: list = None) -> DataFrame:
    """
    Creates an audit DataFrame and writes it to Delta format.

    :param df: Input DataFrame from which unique identifiers are extracted.
    :param unique_identifier_desc: Column name that acts as a unique identifier.
    :param table_name: Name of the source table.
    :param stage_name: Name of the data processing stage.
    :param description: Description of the table.
    :param additional_columns: List of additional columns to include in the audit DataFrame.
    :return: DataFrame containing the audit information.
    """

    dt_desc = datetime.utcnow()

    additional_columns = additional_columns or []  # Default to an empty list if None   
    additional_columns = [col(c) for c in additional_columns if c is not None]  # Filter out None values

    audit_df = df.select(col(unique_identifier_desc).alias("unique_identifier"), *additional_columns) \
        .withColumn("Runid", lit(run_id_value)) \
        .withColumn("Unique_identifier_desc", lit(unique_identifier_desc)) \
        .withColumn("Stage_name", lit(stage_name)) \
        .withColumn("Table_name", lit(table_name)) \
        .withColumn("Run_dt", lit(dt_desc).cast(TimestampType())) \
        .withColumn("Description", lit(description))

    list_cols = audit_df.columns

    final_audit_df = audit_df.groupBy(*list_cols).agg(count("*").cast(IntegerType()).alias("Record_count"))

    # final_audit_df.write.format("delta").mode("append").option("mergeSchema", "true").save(audit_delta_path)
    
    return final_audit_df

In [0]:
# Define Delta Table Path in Azure Storage


if not DeltaTable.isDeltaTable(spark, audit_delta_path):
    print(f"🛑 Delta table '{audit_delta_path}' does not exist. Creating an empty Delta table...")

    # Create an empty DataFrame
    empty_df = spark.createDataFrame([], audit_schema)

    # Write the empty DataFrame in Delta format to create the table
    empty_df.write.format("delta").mode("overwrite").save(audit_delta_path)

    print("✅ Empty Delta table successfully created in Azure Storage.")
else:
    print(f"⚡ Delta table '{audit_delta_path}' already exists.")

In [0]:
audit_params = [
        {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "bronze_adjudicator_et_hc_dnur",
        "stage_name": "bronze_stage",
        "description": "Combines adjudicator data with hearing centre, employment terms, and do not use reason details. Provides a standardized view of adjudicators, their designated centres, employment terms, judicial status, and restrictions. Includes metadata such as source files, modification timestamps, and process tracking for auditing."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "bronze_johistory_users",
        "stage_name": "bronze_stage",
        "description": "Combines JoHistory records with user details, providing historical adjudicator activity along with corresponding user information. Includes comments, user names, audit timestamps, source filenames, and process metadata for tracking and auditing."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "bronze_othercentre_hearingcentre",
        "stage_name": "bronze_stage",
        "description": "Combines OtherCentre records with HearingCentre details, linking adjudicators to their assigned hearing centres. Includes metadata such as timestamps, source files, and process tracking for auditing and traceability."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "bronze_adjudicator_role",
        "stage_name": "bronze_stage",
        "description": "Filters and segments adjudicators, retaining only those who are not assigned roles 7 or 8, or have no assigned role. Uses data from bronze_adjudicator_et_hc_dnur and bronze_adjudicator_role to generate a complete list of adjudicators."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "stg_joh_filtered",
        "stage_name": "segmentation_stage",
        "description": "Filters and segments adjudicators, retaining only those who are not assigned roles 7 or 8, or have no assigned role. Uses data from bronze_adjudicator_et_hc_dnur and bronze_adjudicator_role to generate a complete list of adjudicators."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "silver_adjudicator_detail",
        "stage_name": "silver_stage",
        "description": "Filters adjudicators based on segmentation criteria and enriches their records with Hearing Centre and Do Not Use Reason (DNUR) details. Standardizes key attributes such as correspondence address, contact details, employment terms, and judicial status."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "silver_history_detail",
        "stage_name": "silver_stage",
        "description": "Filters and enhances historical adjudicator activity records by incorporating user details. Maps history types to their corresponding descriptions, providing a structured view of events such as allocations, case updates, and administrative actions."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "silver_othercentre_detail",
        "stage_name": "silver_stage",
        "description": "Filters and enhances OtherCentre records by applying adjudicator segmentation criteria. Retains adjudicators linked to Hearing Centres while ensuring completeness through process metadata, timestamps, and source tracking."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "silver_appointment_detail",
        "stage_name": "silver_stage",
        "description": "Filters and enhances adjudicator role records by mapping role codes to their descriptions. Provides a structured view of adjudicator appointments, including start and end dates, and metadata for auditing."
    },
    {
        "unique_identifier_cols": ["client_identifier"],
        "table_name": "silver_archive_metadata",
        "stage_name": "silver_stage",
        "description": "Metadata table of adjudicator records by combining various metadata fields. Provides a structured view of adjudicator details, including event dates, region, and other relevant information for archival purposes."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "stg_judicial_officer_combined",
        "stage_name": "staging_stage",
        "description": "Metadata table of adjudicator records by combining various metadata fields. Provides a structured view of adjudicator details, including event dates, region, and other relevant information for archival purposes."
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "stg_create_joh_json_content",
        "stage_name": "staging_stage",
        "description": "Generates JSON-formatted adjudicator records for gold-level outputs. Creates structured JSON content for each adjudicator and assigns a filename. Tracks JSON creation status to identify failures and successful transformations",
        "Extra_columns_mapping": {"File_name": "JSONFileName", "Status": "JSONStatus"}
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "stg_create_joh_html_content",
        "stage_name": "staging_stage",
        "description": "Generates HTML-formatted adjudicator records for gold-level outputs. Uses a UDF to transform data into structured HTML content and assigns a filename. Tracks HTML creation status to identify failures and succe",
        "Extra_columns_mapping": {"File_name": "HTMLFileName", "Status": "HTMLStatus"}
    },
    {
        "unique_identifier_cols": ["client_identifier"],
        "table_name": "stg_create_joh_a360_content",
        "stage_name": "staging_stage",
        "description": "Generates A360-formatted adjudicator records for gold-level outputs. Uses a UDF to transform metadata into A360 content and assigns processing statuses. Supports Hive-based retrieval for non-initial loads, ensuring comprehensive archival integration.",
        "Extra_columns_mapping": {"File_name": "NotYetBatched", "Status": "A360Status"}
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "gold_judicial_officer_with_json",
        "stage_name": "Gold_stage",
        "description": "Final gold-level table integrating adjudicator records with validated HTML content. Ensures data integrity by enforcing error-free JSON content. Optimizes processing through repartitioning and triggers upload operations for structured archival and distribution.",
        "Extra_columns_mapping": {"File_name": "JSONFileName", "Status": "UploadStatus"}
    },
    {
        "unique_identifier_cols": ["AdjudicatorId"],
        "table_name": "gold_judicial_officer_with_html",
        "stage_name": "Gold_stage",
        "description": "Final gold-level table integrating adjudicator records with validated JSON content. Ensures data integrity by enforcing error-free HTML content. Optimizes processing through repartitioning and triggers upload operations for structured archival and distribution.",
        "Extra_columns_mapping": {"File_name": "HTMLFileName", "Status": "UploadStatus"}
    },
    {
        "unique_identifier_cols": ["A360BatchId"],
        "table_name": "gold_judicial_officer_with_a360",
        "stage_name": "Gold_stage",
        "description": "Final gold-level table consolidating adjudicator A360 content for structured archival and processing. Ensures data integrity by filtering out records with errors in A360 content. Aggregates and batches records, optimizes processing through repartitioning, and triggers upload operations.",
        "Extra_columns_mapping": {"File_name": "A360FileName", "Status": "UploadStatus"}
    }
]

audit_dataframes = []

for params in audit_params:
    table_name = params["table_name"]
    stage_name = params["stage_name"]
    unique_identifier_cols = params["unique_identifier_cols"]
    description = params["description"]
    extra_columns_mapping = params.get("Extra_columns_mapping", {})
    unique_identifier_desc = "_".join(unique_identifier_cols)

    try:

        df_logging = spark.read.table(f"hive_metastore.ariadm_arm_joh.{table_name}")

        df_audit = df_logging
        if len(unique_identifier_cols) > 1:
            df_audit = df_audit.withColumn(
                unique_identifier_desc, 
                concat_ws("_", *[col(c).cast("string") for c in unique_identifier_cols])
            )
        else:
            df_audit = df_audit.withColumn(unique_identifier_desc, col(unique_identifier_desc))

        # Apply extra column mappings dynamically
        if not extra_columns_mapping:
            df_audit = df_audit.withColumn("File_Name", lit(None)).withColumn("Status", lit(None))
        else:
            for new_col, source_col in extra_columns_mapping.items():
                if source_col == "NotYetBatched":
                    df_audit = df_audit.withColumn(new_col, lit("NotYetBatched"))
                else:
                    df_audit = df_audit.withColumn(new_col, col(source_col))
                        

        # Generate the audit DataFrame
        df_audit_appended = create_audit_df(
            df_audit,
            unique_identifier_desc=unique_identifier_desc,
            table_name=table_name,
            stage_name=stage_name,
            description=description,
            additional_columns = ["File_Name", "Status"]
        )

        audit_dataframes.append(df_audit_appended)

        print(f"✅ Successfully processed table: {table_name}")

    except Exception as e:
        print(f"🛑 Failed to process table: {table_name}. Error: {str(e)}")
        failed_table = f"Table {table_name} does not exist"


        # Table does not exist, create an audit entry for it
        status = f"Failed - Table {table_name} does not exist"

        row_data = {
            "Runid": run_id_value,
            "Unique_identifier_desc": unique_identifier_desc,
            "Unique_identifier": None,
            "Table_name": table_name,
            "Stage_name": stage_name,
            "Record_count": 0,
            "Run_dt": datetime.now(),
            "Batch_id": None,
            "Description": description,
            "File_name": None,
            "Status": status
        }

        row_df = spark.createDataFrame([row_data], schema=audit_schema)
        audit_dataframes.append(row_df)




df_final_audit = audit_dataframes[0]
for df in audit_dataframes[1:]:
    df_final_audit = df_final_audit.unionByName(df, allowMissingColumns=True)



In [0]:
df_final_audit.write.format("delta").mode("append").option("mergeSchema", "true").save(audit_delta_path)

In [0]:
dbutils.notebook.exit("Notebook completed successfully")

## Appendix

In [0]:
# df_final_audit.createOrReplaceTempView("tv_final_audit")

In [0]:

# %sql
# select * from tv_final_audit
# where table_name like 'gold_judicial_officer_with_json%'

In [0]:
%sql
select * from hive_metastore.ariadm_arm_joh.gold_judicial_officer_with_json